In [1]:
import numpy as np
import imutils
import time
import cv2
import os

In [2]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream

2022-04-05 23:10:17.819629: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/harsh/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-04-05 23:10:17.819695: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def mask_model(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),(104.0, 177.0, 123.0))
    faceNet.setInput(blob)
    detections = faceNet.forward()
    faces = []
    locs = []
    preds = []
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))


    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)

    return (locs, preds)

In [4]:
!pwd

/home/harsh/DSAT/mask-detection


In [5]:
prototxtPath = "/home/harsh/DSAT/mask-detection/deploy.prototxt"
weightsPath = "/home/harsh/DSAT/mask-detection/res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [6]:
maskNet = load_model("/home/harsh/DSAT/mask-detection/mask_model")

2022-04-05 23:10:38.709890: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/harsh/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-04-05 23:10:38.709972: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-05 23:10:38.710008: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ubuntu-2110): /proc/driver/nvidia/version does not exist
2022-04-05 23:10:38.710606: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
vs = VideoStream(src=0).start()

In [ ]:
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    (locs, preds) = mask_model(frame, faceNet, maskNet)
    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        cv2.putText(frame, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cv2.destroyAllWindows()
vs.stop()

QObject::moveToThread: Current thread (0x561d47422e90) is not the object's thread (0x561d48e16ba0).
Cannot move to target thread (0x561d47422e90)

QObject::moveToThread: Current thread (0x561d47422e90) is not the object's thread (0x561d48e16ba0).
Cannot move to target thread (0x561d47422e90)

QObject::moveToThread: Current thread (0x561d47422e90) is not the object's thread (0x561d48e16ba0).
Cannot move to target thread (0x561d47422e90)

QObject::moveToThread: Current thread (0x561d47422e90) is not the object's thread (0x561d48e16ba0).
Cannot move to target thread (0x561d47422e90)

QObject::moveToThread: Current thread (0x561d47422e90) is not the object's thread (0x561d48e16ba0).
Cannot move to target thread (0x561d47422e90)

QObject::moveToThread: Current thread (0x561d47422e90) is not the object's thread (0x561d48e16ba0).
Cannot move to target thread (0x561d47422e90)

QObject::moveToThread: Current thread (0x561d47422e90) is not the object's thread (0x561d48e16ba0).
Cannot move to tar

QObject::moveToThread: Current thread (0x561d47422e90) is not the object's thread (0x561d48e16ba0).
Cannot move to target thread (0x561d47422e90)

QObject::moveToThread: Current thread (0x561d47422e90) is not the object's thread (0x561d48e16ba0).
Cannot move to target thread (0x561d47422e90)

